# 타이타닉 생존자 예측

In [9]:
import numpy as np
import pandas as pd
import seaborn as sns

In [10]:
df = sns.load_dataset('titanic')
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


### 1. 데이터 전처리

- feature selection

In [11]:
df = df[['survived', 'pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked', 'deck']]
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,deck
0,0,3,male,22.0,1,0,7.2500,S,NaN
1,1,1,female,38.0,1,0,71.2833,C,C
2,1,3,female,26.0,0,0,7.9250,S,NaN


- 결측치 처리

In [12]:
# 결측치 확인
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
sibsp         0
parch         0
fare          0
embarked      2
deck        688
dtype: int64

In [13]:
# age 컬럼은 평균으로 대체
df.age.fillna(df.age.mean(), inplace=True)
df.age.isna().sum()

0

In [14]:
# embarked 컬럼은 최빈값으로 대체
df.embarked.value_counts()

S    644
C    168
Q     77
Name: embarked, dtype: int64

In [15]:
df.embarked.fillna('S', inplace=True)
df.embarked.isna().sum()

0

In [16]:
# deck 컬럼은 삭제
df.drop(columns=['deck'], inplace=True)
df.isna().sum().sum()

0

- 카테고리 값인 sex, embarked 컬럼은 숫자로 변환

In [17]:
#LabelEncoder로 변환
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [19]:
df.sex=  le.fit_transform(df.sex)
df.embarked = le.fit_transform(df.embarked)
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


### 2. Train/Test dataset으로 분리

In [20]:
# X와 y를 넘파이 배열로
X = df.iloc[:,1:].values
y = df.survived.values
X.shape, y.shape

((891, 7), (891,))

In [21]:
# y값의 분포
df.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [23]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=2021
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape
# X, y 개수 안맞는데, 그거 개수 맞춰주는 게 stratify !

((712, 7), (179, 7), (712,), (179,))

In [24]:
np.unique(y_train, return_counts=True)

(array([0, 1]), array([439, 273]))

In [25]:
print(342*439, 549*273)

150138 149877


### 3. RandomForest 모델로 학습

In [28]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=2021)
rfc.get_params() 
# 결과 보니까, 'n_estimators': 100, 예측기 100개 있넹

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': 2021,
 'verbose': 0,
 'warm_start': False}

In [29]:
# RFC로 학습
rfc.fit(X_train, y_train)

RandomForestClassifier(random_state=2021)

### 4. 모델 예측 및 평가

In [32]:
# pred = rfc.predict(X_test)
rfc.score(X_test, y_test)
# 81% 정도 맞힌다는 뜻

0.8100558659217877

### 5. 3,4 대신에 GridSearchCV 수행

In [35]:
params = {
    'max_depth': [2,4,6,8],
    'min_samples_split': [2,4,6]
}

In [38]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [2, 4, 6, 8],
                         'min_samples_split': [2, 4, 6]},
             scoring='accuracy')

In [39]:
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [41]:
params = {
    'max_depth': [7,8,9,10],
    'min_samples_split': [3,4,5]
}

In [42]:
from sklearn.model_selection import GridSearchCV
grid_rf = GridSearchCV(rfc, param_grid=params, scoring='accuracy', cv=5)
grid_rf.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=2021),
             param_grid={'max_depth': [7, 8, 9, 10],
                         'min_samples_split': [3, 4, 5]},
             scoring='accuracy')

In [43]:
grid_rf.best_params_

{'max_depth': 8, 'min_samples_split': 4}

In [44]:
best_rf = grid_rf.best_estimator_
best_rf.score(X_test, y_test)

0.8100558659217877

### 6. 테스트 데이터에 적용

In [45]:
X_test[27], y_test[27]

(array([ 3.    ,  1.    , 32.    ,  0.    ,  0.    , 56.4958,  2.    ]), 1)

In [46]:
grid_rf.predict(X_test[27].reshape(1,-1))

array([0])

In [47]:
grid_rf.predict(X_test[12].reshape(1,-1)), y_test[12]

(array([0]), 0)

In [49]:
X_test[12] # 12,27 이런 게 내가 넣는 index 값

array([ 2. ,  1. , 16. ,  0. ,  0. , 10.5,  2. ])

### 7. 엉터리 분류기
- 여성이면 생존이라 예측, 그 외의 경우는 사망

In [52]:
df.groupby('sex')['survived'].mean()

sex
0    0.742038
1    0.188908
Name: survived, dtype: float64

In [54]:
df.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2


In [55]:
from sklearn.base import BaseEstimator

class MyClassifier(BaseEstimator):
    # fit(), predict() method만 재정의(override)
    def fit(self, X, y):
        pass
    def predict(self, X):
        pred = np.zeros(X.shape[0]) # 179번 돌린다.
        for i in range(X.shape[0]):
            if X[i,1] == 0: # pclass부터가 0번째 열이니까
                pred[i] = 1 # 여성이면 생존으로 처리 (i는 survived니까)
        return pred

In [56]:
my_cf = MyClassifier()
my_cf.fit(X_train, y_train)
pred_my = my_cf.predict(X_test)

In [57]:
X_test[:5]

array([[  3.        ,   0.        ,  38.        ,   1.        ,
          5.        ,  31.3875    ,   2.        ],
       [  3.        ,   1.        ,  29.69911765,   0.        ,
          0.        ,  56.4958    ,   2.        ],
       [  1.        ,   0.        ,  14.        ,   1.        ,
          2.        , 120.        ,   2.        ],
       [  2.        ,   1.        ,  29.69911765,   0.        ,
          0.        ,   0.        ,   2.        ],
       [  3.        ,   0.        ,  18.        ,   0.        ,
          0.        ,   6.75      ,   1.        ]])

In [60]:
X_test[:5,1],pred_my[:5]
# 여성 0 이면 생존 1 / 남성 1 이면 사망 0으로 잘 나옴

(array([0., 1., 0., 1., 0.]), array([1., 0., 1., 0., 1.]))

In [61]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_my)

0.7877094972067039

In [63]:
pred_rf = best_rf.predict(X_test)
accuracy_score(y_test, pred_rf)

0.8100558659217877

In [64]:
sdf = pd.DataFrame({'y_test':y_test, 'RF':pred_rf, 'My':pred_my})
sdf.head() # random은 2개 my는 3개 맞힘ㅋ 고생해서 만들었는데 대충한 게 더 잘맞넹 !
# 그러므로, 평가할 때 accuracy_score 너무 믿지 말라

,y_test,RF,My
0,1,0,1.0
1,1,0,0.0
2,1,1,1.0
3,0,0,0.0
4,0,1,1.0


- 모델의 성능을 평가할 때 무조건적으로 정확도를 사용하는 것은 지양하세요..!

In [65]:
from sklearn.metrics import confusion_matrix, precision_score, recall_score
confusion_matrix(y_test, pred_rf)

array([[98, 12],
       [22, 47]])

In [66]:
confusion_matrix(y_test, pred_my)
# negative인거 negative로 예측한거.... 35페이지

array([[95, 15],
       [23, 46]])

In [68]:
precision_score(y_test, pred_my), recall_score(y_test, pred_my)

(0.7540983606557377, 0.6666666666666666)